# Reproduction of Kidney Failure Risk Equation (KFRE)

## Import Libraries

In [1]:
########################### Import Requisite Libraries #########################
import pandas as pd
import numpy as np
import os
import sys
import pprint

################################################################################
# Add the parent directory to sys.path to access 'functions.py'
sys.path.append(os.path.join(os.pardir))
from python_scripts.functions import *
from kfre import add_kfre_risk_col  # import from kfre

## Read in the Data

In [2]:
base_path = os.path.join(os.pardir)

# Go up one level from 'notebooks' to the parent directory, then into the 'data' folder
data_path = os.path.join(os.pardir, "data")

image_path_png = os.path.join(base_path, "images", "png_images")
image_path_svg = os.path.join(base_path, "images", "svg_images")

# Use the function to ensure the 'data' directory exists
ensure_directory(data_path)
ensure_directory(image_path_png)
ensure_directory(image_path_svg)

# Read the CSV file into a DataFrame
df = pd.read_parquet(os.path.join(data_path, "df.parquet"))
df_eda = pd.read_parquet(os.path.join(data_path, "df_eda.parquet"))

Directory exists: ..\data
Directory exists: ..\images\png_images
Directory exists: ..\images\svg_images


In [3]:
# join back sex to original dataframe for eda dataframe so it can be used
df = df.join(df_eda["SEX"], on="Patient_ID", how="inner")

In [4]:
df_kfre = df.copy(deep=True)

In [5]:
df_kfre.columns

Index(['Age', 'Diabetes (1=yes; 0=no)', 'Hypertension (1=yes; 0=no)',
       'Renal disease (DM=1, HTN=2, GN=3, ADPKD=4, Other=5)', 'eGFR-EPI',
       'uPCR', 'Calcium (mmol/L)', 'Phosphate (mmol/L)',
       'Bicarbonate (mmol/L)', 'Albumin (g/l)', 'Follow-up YEARS', 'RIP',
       'ESRD', 'SEX_Female', 'SEX_Male',
       'ETHNICITY_AFRICAN (BLACK OR BLACK BRITISH)',
       'ETHNICITY_ANY OTHER ASIAN BACKGROUND',
       'ETHNICITY_ANY OTHER BLACK BACKGROUND',
       'ETHNICITY_ANY OTHER ETHNIC GROUP',
       'ETHNICITY_ANY OTHER MIXED BACKGROUND',
       'ETHNICITY_ANY OTHER WHITE BACKGROUND',
       'ETHNICITY_BANGLADESHI (ASIAN OR ASIAN BRITISH)',
       'ETHNICITY_CARIBBEAN (BLACK OR BLACK BRITISH)',
       'ETHNICITY_CHINESE (OTHER ETHNIC GROUPS)',
       'ETHNICITY_INDIAN (ASIAN OR ASIAN BRITISH)', 'ETHNICITY_NOT STATED',
       'ETHNICITY_PAKISTANI (ASIAN OR ASIAN BRITISH)',
       'ETHNICITY_WHITE AND BLACK AFRICAN (MIXED)', 'ETHNICITY_WHITE BRITISH',
       'ETHNICITY_WHITE IRIS

In [6]:
# subset the dataframe to include only relevant variables for the KFRE equation
# Age, eGFR, log-normalized uACR, Diabetes, Hypertension
df_kfre = df_kfre[
    [
        "Age",
        "SEX",
        "eGFR-EPI",
        "uACR",
        "Diabetes (1=yes; 0=no)",
        "Hypertension (1=yes; 0=no)",
        "Albumin_g_dl",
        "Calcium_mg_dl",
        "Phosphate_mg_dl",
        "Bicarbonate (mmol/L)",
        "ESRD",
    ]
]

In [7]:
df_kfre["Region"] = "Non North American"

In [8]:
df = add_kfre_risk_col(
    df=df_kfre,
    age_col="Age",
    sex_col="SEX",
    eGFR_col="eGFR-EPI",
    uACR_col="uACR",
    dm_col="Diabetes (1=yes; 0=no)",
    htn_col="Hypertension (1=yes; 0=no)",
    albumin_col="Albumin_g_dl",
    phosphorous_col="Phosphate_mg_dl",
    bicarbonate_col="Bicarbonate (mmol/L)",
    calcium_col="Calcium_mg_dl",
    num_vars=(4, 6, 8),
    years=(2, 5),
    is_north_american=False,
    copy=False,  # Modify the original DataFrame directly
)
# The resulting DataFrame 'df_kfre' now includes new columns with risk
# predictions for each model and time frame

In [9]:
df_kfre.head()

,Age,SEX,eGFR-EPI,uACR,Diabetes (1=yes; 0=no),Hypertension (1=yes; 0=no),Albumin_g_dl,Calcium_mg_dl,Phosphate_mg_dl,Bicarbonate (mmol/L),ESRD,Region,kfre_4var_2year,kfre_4var_5year,kfre_6var_2year,kfre_6var_5year,kfre_8var_2year,kfre_8var_5year
Patient_ID,,,,,,,,,,,,,,,,,,
568268916,87.24,Male,19.0,102.438624,1.0,1.0,3.7,11.12,2.728,27.2,0,Non North American,0.067174,0.236057,0.064034,0.222093,0.029092,0.124367
659549633,56.88,Female,15.0,1762.039423,0.0,1.0,3.0,9.72,3.162,21.3,1,Non North American,0.449508,0.900886,0.485479,0.919696,0.413627,0.909392
406748956,66.53,Female,17.0,659.136129,0.0,1.0,3.6,9.32,3.844,27.8,0,Non North American,0.219611,0.617170,0.240018,0.647115,0.157298,0.536920
109228713,69.92,Male,12.0,1145.245058,0.0,1.0,3.9,9.16,5.580,20.7,1,Non North American,0.482800,0.922156,0.520907,0.938744,0.599700,0.983730
220533110,81.14,Female,15.0,980.939665,1.0,1.0,4.3,9.80,4.309,26.2,0,Non North American,0.235649,0.646748,0.226075,0.621908,0.147547,0.512324


## Validate the KFRE

We can validate the KFRE for each individual patient using an online calculator like the one below:

https://qxmd.com/calculate/calculator_308/kidney-failure-risk-equation-4-variable

We can also validate the medians of the 4 variable, 6 variable, and 8 variable equations for the 2 year and 5 year outcomes, as shown below.

In [10]:
med_2_year_4var = df_kfre["kfre_4var_2year"].median()
print(f"Median of 4 variable, 2 year KFRE = {med_2_year_4var*100:.0f}%")

Median of 4 variable, 2 year KFRE = 23%


Ali et al. show a median value of 24 % for the 4 variable 2-year outcome.

In [11]:
med_5_year_4var = df_kfre["kfre_4var_5year"].median()
print(f"Median of 4 variable, 5 year KFRE = {med_5_year_4var*100:.0f}%")

Median of 4 variable, 5 year KFRE = 64%


Ali et al. show a median value of 65 % for the 4 variable 5-year outcome.

In [12]:
med_2_year_8var = df_kfre["kfre_8var_2year"].median()
print(f"Median of 8 variable, 2 year KFRE = {med_2_year_8var*100:.0f}%")

Median of 8 variable, 2 year KFRE = 20%


Ali et al. show a median value of 20% for the 8 variable 2-year outcome.

In [13]:
med_5_year_8var = df_kfre["kfre_8var_5year"].median()
print(f"Median of 8 variable, 5 year KFRE = {med_5_year_8var*100:.0f}%")

Median of 8 variable, 5 year KFRE = 64%


Ali et al. show a median value of 64% for the 8 variable 5-year outcome.

While the median percentages for the 4-variable 2-year and 5-year outcomes differ by 1%, the median percentages for the 8-variable equations are identical. These discrepancies can be attributed to rounding differences.

## References

Ali, I., Donne, R. L., & Kalra, P. A. (2021). A validation study of the kidney failure risk equation in advanced chronic kidney disease according to disease aetiology with evaluation of discrimination, calibration and clinical utility. *BMC Nephrology, 22(1),* 194.  doi: 10.1186/s12882-021-02402-1

Sumida K, Nadkarni GN, Grams ME, Sang Y, Ballew SH, Coresh J, Matsushita K, Surapaneni A, Brunskill N, Chadban SJ, Chang AR, Cirillo M, Daratha KB, Gansevoort RT, Garg AX, Iacoviello L, Kayama T, Konta T, Kovesdy CP, Lash J, Lee BJ, Major RW, Metzger M, Miura K, Naimark DMJ, Nelson RG, Sawhney S, Stempniewicz N, Tang M, Townsend RR, Traynor JP, Valdivielso JM, Wetzels J, Polkinghorne KR, Heerspink HJL, for the Chronic Kidney Disease Prognosis Consortium. (2020). Conversion of urine protein-creatinine ratio or urine dipstick protein to urine albumin-creatinine ratio for use in chronic kidney disease screening and prognosis. *Ann Intern Med,* ***173***(6), 426-435. doi: 10.7326/M20-0529.

Tangri, N., Stevens, L. A., Griffith, J., Tighiouart, H., Djurdjev, O., Naimark, D., Levin, A., & Levey, A. S. (2011). *A predictive model for progression of chronic kidney disease to kidney failure. JAMA,* **305**(15), 1553-1559. doi: 10.1001/jama.2011.451.  

Tangri N, Grams ME, Levey AS, Coresh J, Appel LJ, Astor BC, Chodick G, Collins AJ, Djurdjev O, Elley CR, Evans M, Garg AX, Hallan SI, Inker LA, Ito S, Jee SH, Kovesdy CP, Kronenberg F, Heerspink HJL, Marks A, Nadkarni GN, Navaneethan SD, Nelson RG, Titze S, Sarnak MJ, Stengel B, Woodward M, Iseki K, for the CKD Prognosis Consortium. (2016). *Multinational assessment of accuracy of equations for predicting risk of kidney failure: A meta-analysis. JAMA,* **315**(2), 164–174. doi: 10.1001/jama.2015.18202.

